In [33]:
import pandas as pd
import requests
import json
import numpy as np
import plotly.express as px
from dataclasses import dataclass, field
from typing import Dict, Literal
from datetime import datetime
import random
from scipy.spatial import cKDTree


# 1. Network Emission Rate $E_s(\tau)$

In [248]:
SupportedTickers = Literal['eth', 'btc', 'matic']
@dataclass
class CCRIApi:
    """
    Carbon Crypto Ratings Initiative API
    """
    key: str = 'dSX0OWkLVpTondd5WZPjFEOt'
    headers: Dict[str,str] = field(default_factory=lambda: {"accept": "application/json"})
    base_url: str = 'https://v2.api.carbon-ratings.com'

    def get_ticker_info(self, ticker: SupportedTickers):
        response = requests.get(f"{self.base_url}/currencies/{ticker}?key={self.key}", headers=self.headers)
        return response.json()
        
    def get_attributed_emissions(self, ticker, input_data_type, allocation_strategy) -> pd.DataFrame:
        url = f"{self.base_url}/currencies/{ticker}/emissions/{input_data_type}/{allocation_strategy}?key={self.key}"
        response = requests.get(url, headers=self.headers)
        return response.json()
    
    def get_ticker_emissions(self, ticker: SupportedTickers) -> pd.DataFrame:
        """
        Get the historical (daily and annualized emissions for a given ticker
        :param ticker: str - ticker of the cryptocurrency in lowercase
                       matic, algo, dash, bnb, doge, sol, eth, bsv, ada, eth2, xtz, atom, bch, btc, ltc, trx, dot, avax
        :return: pd.DataFrame - emissions data for the ticker
                 emissions_24h: The estimated emissions for the given network in the 24 hours on the given date, in metaData->outputUnit
                 emissions_365d: The estimated emissions for the given network annualized for the given date, in metaData->outputUnit
                 intensity: The estimated carbon intensity of the given network on the given date, in kilogram of CO2 equivalent per kilowatt hour (kgCO2e/kWh)
        """
        url = f"{self.base_url}/currencies/{ticker}/emissions/network?key={self.key}"
        response = requests.get(url, headers=self.headers)
        ticker_info = self.get_ticker_info(ticker)
        data = None
        try:
            data = response.json()
        except json.decoder.JSONDecodeError:
            return response.text
        
        if response.status_code == 200:
            df = pd.DataFrame(data['entries'])
            emission_unit = data['metaData']['outputUnit']
            df.rename(columns={'emissions_24h': f'emissions_24h_{emission_unit}', 'emissions_365d': f'emissions_365d_{emission_unit}', 'intensity': 'intesity_kgCO2e/kWh'}, inplace=True)
            return pd.DataFrame(df)
        else:
            return response.text
        
    def refresh_emissions_csv(self, ticker: SupportedTickers) -> None:
        data = self.get_ticker_emissions(ticker)
        data.to_csv(f'./data/CCRI/ccri_{ticker}_emissions.csv', index=False)
        
    def refresh_all_emissions_csv(self) -> None:
        for ticker in ['eth', 'btc', 'eth2']:
            self.refresh_emissions_csv(ticker)
        
        
        

In [249]:
emissions_eth = CCRIApi().refresh_all_emissions_csv()

In [250]:
emissions_eth.to_csv('./data/ccri_eth_emissions.csv', index=False)

AttributeError: 'NoneType' object has no attribute 'to_csv'

In [252]:
emissions_eth = pd.read_csv('./data/CCRI/ccri_eth_emissions.csv')
emissions_eth_pos = pd.read_csv('./data/CCRI/ccri_eth2_emissions.csv')
emissions_btc = pd.read_csv('./data/CCRI/ccri_btc_emissions.csv')
        

In [253]:
px.line(emissions_btc, x='date', y='emissions_24h_mt', title='Bitcoin 24h Emissions')

In [254]:
emissions_btc.isna().sum()

date                   0
emissions_24h_mt       0
emissions_365d_mt      0
intesity_kgCO2e/kWh    0
dtype: int64

# 2. Emission Attribution Parameters $T_s(\tau)$, $G_s(\tau)$, $B_s(\tau)$

https://github.com/shawntabrizi/ethgraph

In [255]:
@dataclass

class EtherscanAPI:
    key: str = 'G54GI11UWC84RPJ9PS7E32M2D26RF3DYAV'

## 2.1 Mock Data
### 2.1.1 Bitcoin User Mock Data

In [340]:
btc_price_df = pd.read_csv('./data/btc_historical_price.csv')


In [449]:

def find_nearest_date(target_date, date_series):
    return pd.to_datetime(date_series).iloc[(pd.to_datetime(date_series) - target_date).abs().argsort()[:1]].values[0]

def generate_btc_persona_data(persona, start_date, end_date, btc_price_df, initial_investment_usd=1000):
    df = pd.DataFrame(columns=['date', 'xpub_address', 'balance_btc', 'num_transactions'])
    
    # Find the nearest historical BTC price to the intended start date to set the initial balance
    nearest_start_date = find_nearest_date(start_date, btc_price_df['date'])
    initial_price = btc_price_df[pd.to_datetime(btc_price_df['date']) == nearest_start_date]['price'].values[0]
    initial_balance = initial_investment_usd / initial_price
    
    current_balance = initial_balance
    xpub_address = f"{random.randint(10000, 99999)}"
    
    prev_price = initial_price  # For tracking volatility
    
    for date, price in zip(btc_price_df['date'], btc_price_df['price']):
        date_dt = datetime.strptime(date, '%m/%d/%Y %H:%M:%S')
        
        if date_dt < start_date or date_dt > end_date:
            continue
            
        # Calculate volatility as percentage change in price
        volatility = abs((price - prev_price) / prev_price)
        
        delta_balance = 0
        num_transactions = 0
        delta_balance_usd = 0
        
        if persona == 'Hodler':
            num_transactions = np.random.poisson(0.1)
            delta_balance_usd = np.random.normal(0.05, 0.01) * current_balance * price  # Scale by current balance
        elif persona == 'Day Trader':
            num_transactions = np.random.poisson(5 * (1 + volatility))  # Increase frequency with volatility
            delta_balance_usd = np.random.normal(50, 500)  # 50 USD with a standard deviation of 10
        elif persona == 'Retail Payment User':
            num_transactions = np.random.poisson(5)
            delta_balance_usd = np.random.normal(30, 40)  # 5 USD with a standard deviation of 1
        elif persona == 'Institutional Investor':
            num_transactions = np.random.poisson(5)
            delta_balance_usd = np.random.uniform(-0.005, 0.02) * current_balance * price  # 4000 USD with a standard deviation of 2000
        elif persona == 'Occasional User':
            num_transactions = np.random.poisson(0.2)
            delta_balance_usd = np.random.normal(20, 10)  # 20 USD with a standard deviation of 10
        
        # Convert delta_balance from USD to BTC
        delta_balance = delta_balance_usd / price
        
        # Update the current balance
        current_balance += delta_balance * num_transactions
        networth_usd = current_balance * price
        
        # Ensure the balance never goes negative
        current_balance = max(0, current_balance)
        networth_usd = max(0, networth_usd)
        
        # Append the data for this week to the dataframe
        df = pd.concat([df, pd.DataFrame({
            'date': [date],
            'xpub_address': [xpub_address],
            'balance_btc': [current_balance],
            'num_transactions': [num_transactions],
            'networth_usd': [networth_usd],
            'first_seen': [start_date],
            'persona_name': [persona]
        })], ignore_index=True)
        
        # Update prev_price for next iteration
        prev_price = price
    
    return df



# Generate further improved mock data for each persona, assuming they all started at different times
further_improved_persona_list = [
    ('The Hodler', datetime(2015, 1, 1), datetime(2023, 6, 30), 1000),
    ('Day Trader', datetime(2016, 1, 1), datetime(2023, 6, 30), 2000),
    ('Retail Payment User', datetime(2017, 1, 1), datetime(2023, 6, 30), 500),
    ('Institutional Investor', datetime(2018, 1, 1), datetime(2023, 6, 30), 20000),
    ('Occasional User', datetime(2020, 11, 27), datetime(2023, 6, 30), 300),
]

btc_mock_dfs = []

for persona, start_date, end_date, initial_investment_usd in further_improved_persona_list:
    mock_df = generate_btc_persona_data(persona, start_date, end_date, btc_price_df, initial_investment_usd)
    btc_mock_dfs.append(mock_df)

# Concatenate all the persona dataframes into one
btc_mock_data = pd.concat(btc_mock_dfs, ignore_index=True)
btc_mock_data.head(), btc_mock_data.tail()
btc_mock_data.to_csv('./data/users/btc_mock_data.csv', index=False)


In [450]:
btc_mock_data

,date,xpub_address,balance_btc,num_transactions,networth_usd,first_seen,persona_name
0,11/20/2015 23:58:00,66170,3.058104,0,1000.000000,2015-01-01,The Hodler
1,11/27/2015 23:58:00,66170,3.058104,0,1092.477064,2015-01-01,The Hodler
2,12/4/2015 23:58:00,66170,3.058104,0,1186.513761,2015-01-01,The Hodler
3,12/11/2015 23:58:00,66170,3.058104,0,1335.993884,2015-01-01,The Hodler
4,12/18/2015 23:58:00,66170,3.058104,0,1410.672783,2015-01-01,The Hodler
...,...,...,...,...,...,...,...
1578,5/26/2023 23:58:00,91744,0.034405,0,924.380187,2020-11-27,Occasional User
1579,6/2/2023 23:58:00,91744,0.034405,0,931.594838,2020-11-27,Occasional User
1580,6/9/2023 23:58:00,91744,0.034405,0,889.569582,2020-11-27,Occasional User
1581,6/16/2023 23:58:00,91744,0.034405,0,911.946354,2020-11-27,Occasional User


In [451]:
btc_mock_data.groupby('persona_name').agg({'balance_btc': 'mean', 'networth_usd': 'mean', 'first_seen': 'min', 'num_transactions': 'sum'})

,balance_btc,networth_usd,first_seen,num_transactions
persona_name,,,,
Day Trader,48.847781,7.946545e+05,2016-01-01,2066
Institutional Investor,18347.759114,4.775758e+08,2018-01-01,1490
Occasional User,0.025889,8.765961e+02,2020-11-27,25
Retail Payment User,6.718821,1.386536e+05,2017-01-01,1741
The Hodler,3.058104,4.687002e+04,2015-01-01,0


# 2.1.2 Ethereum User Mock Data

In [473]:
# Helper function to generate mock data for an Ethereum persona, updated to implement strategies 1, 2, and 4
def generate_eth_persona_data(persona, start_date, end_date, eth_price_df, initial_investment_usd=1000):
    df = pd.DataFrame(columns=['date', 'account_address', 'balance_eth', 'networth_usd', 'gas_spent'])
    
    # Find the nearest historical ETH price to the intended start date to set the initial balance
    nearest_start_date = find_nearest_date(start_date, eth_price_df['Date'])
    initial_price = eth_price_df[pd.to_datetime(eth_price_df['Date']) == nearest_start_date]['Close'].values[0]
    initial_balance = initial_investment_usd / initial_price
    
    current_balance = initial_balance
    networth_usd = initial_investment_usd  # Initially, net worth is the same as the investment
    account_address = f"0x{persona[:3]}_{random.randint(1000, 9999)}"
    
    prev_price = initial_price  # For tracking volatility
    
    for date, price in zip(eth_price_df['Date'], eth_price_df['Close']):
        date_dt = datetime.strptime(date, '%m/%d/%Y %H:%M:%S')
        
        if date_dt < start_date or date_dt > end_date:
            continue
        
        
        # Calculate volatility as percentage change in price
        volatility = abs((price - prev_price) / prev_price)
        
        delta_balance, num_transactions, gas_spent = 0, 0, 0
        delta_balance_usd = 0
        
        if persona == 'The Hodler':
            num_transactions = np.random.poisson(0.1)
            delta_balance_usd = np.random.normal(0.05, 0.01) * current_balance * price  # Scale by current balance
        elif persona == 'Day Trader':
            num_transactions = np.random.poisson(5 * (1 + volatility))  # Increase frequency with volatility
            delta_balance_usd = np.random.normal(50, 500)  # 50 USD with a standard deviation of 10
        elif persona == 'DAO Lending Protocol':
            num_transactions = np.random.poisson(50)
            delta_balance_usd = np.random.normal(500, 200)  # 500 USD with a standard deviation of 200
        elif persona == 'Retail Payment User':
            num_transactions = np.random.poisson(5)
            delta_balance_usd = np.random.normal(30, 40)  # 5 USD with a standard deviation of 1
        elif persona == 'Institutional Investor':
            num_transactions = np.random.poisson(5)
            delta_balance_usd =np.random.uniform(-0.005, 0.02) * current_balance * price   # 1000 USD with a standard deviation of 400
        elif persona == 'Occasional User (PoS)':
            num_transactions = np.random.poisson(5)
            delta_balance_usd = np.random.normal(10, 5)  # 10 USD with a standard deviation of 5
        
        # Convert delta_balance from USD to ETH
        delta_balance = delta_balance_usd / price
        
        # Update the current balance and networth
        current_balance += delta_balance * num_transactions
        networth_usd = current_balance * price  # Assumes all assets held are in ETH for simplicity
        
        # Ensure the balance and networth never go negative
        current_balance = max(0, current_balance)
        networth_usd = max(0, networth_usd)
        
        # Gas spent
        gas_spent = num_transactions * random.randint(21000, 50000)  # Gas for simple transfers
        
        # Append the data for this week to the dataframe
        df = pd.concat([df, pd.DataFrame({
            'date': [date],
            'account_address': [account_address],
            'balance_eth': [current_balance],
            'networth_usd': [networth_usd],
            'gas_spent_addr': [gas_spent],
            'first_seen': [start_date],
            'persona_name': [persona]
        })], ignore_index=True)
        
        # Update prev_price for next iteration
        prev_price = price
    
    return df    
# Re-run the Ethereum mock data generation with the corrected column names
eth_mock_data_frames = []

eth_persona_list = [
    ('The Hodler', datetime(2015, 1, 1), datetime(2023, 6, 30), 1000),
    ('Day Trader', datetime(2016, 1, 1), datetime(2023, 6, 30), 2000),
    ('DAO Lending Protocol', datetime(2020, 1, 1), datetime(2023, 6, 30), 20000),
    ('Retail Payment User', datetime(2017, 1, 1), datetime(2023, 6, 30), 500),
    ('Institutional Investor', datetime(2018, 1, 1), datetime(2023, 6, 30), 20000),
    ('Occasional User (PoS)', datetime(2022, 9, 25), datetime(2023, 6, 30), 700),
]

eth_price_df = pd.read_csv('./data/eth_historical_price.csv')

for persona, start_date, end_date, initial_investment_usd in eth_persona_list:
    mock_df = generate_eth_persona_data(persona, start_date, end_date, eth_price_df, initial_investment_usd)
    eth_mock_data_frames.append(mock_df)

eth_mock_data = pd.concat(eth_mock_data_frames, ignore_index=True)
eth_mock_data.head()
eth_mock_data.to_csv('./data/users/eth_mock_data.csv', index=False)


## 3 Weighting factors
### 3.1 Historical Fees
https://blockchair.com/api/plans


In [474]:
# Helper function to calculate weekly relative change for a given series
def calculate_weekly_relative_change(series):
    weekly_series = series.resample('W').last()  # Resample to get weekly data
    return (weekly_series - weekly_series.shift(1)) / weekly_series.shift(1)


In [475]:
with open("./data/btc_market_cap.json", "r") as f:
    btc_market_cap_data = json.load(f)
btc_market_cap_new_transformed = pd.DataFrame(btc_market_cap_data['market_caps'], columns=['date', 'market_cap'])
btc_market_cap_new_transformed['date'] = pd.to_datetime(btc_market_cap_new_transformed['date'], unit='ms')


# Code to load the Ethereum market cap data from the CSV file
eth_market_cap_df = pd.read_csv('./data/eth_market_cap.csv')

# Code to load the Bitcoin average transaction fee data from the TSV file
btc_avg_tx_fee_df = pd.read_csv('./data/btc_avg_tx_fee.tsv', delimiter='\t')

# Code to load the Ethereum average transaction fee data from the TSV file
eth_avg_tx_fee_df = pd.read_csv('./data/eth_avg_simple_tx_fee.tsv', delimiter='\t')

In [476]:
btc_market_cap_new_transformed

,date,market_cap
0,2013-04-28 00:00:00,1.500518e+09
1,2013-04-29 00:00:00,1.575032e+09
2,2013-04-30 00:00:00,1.501657e+09
3,2013-05-01 00:00:00,1.298952e+09
4,2013-05-02 00:00:00,1.148668e+09
...,...,...
3770,2023-08-26 00:00:00,5.070079e+11
3771,2023-08-27 00:00:00,5.060861e+11
3772,2023-08-28 00:00:00,5.078357e+11
3773,2023-08-29 00:00:00,5.083507e+11


In [477]:

# Update the function to take separate date columns for market_cap and tx_fee DataFrames
def calculate_weights(market_cap_df, tx_fee_df, date_col_market, date_col_tx, market_cap_col, tx_fee_col):
    # Convert to datetime and set as index
    market_cap_df[date_col_market] = pd.to_datetime(market_cap_df[date_col_market])
    tx_fee_df[date_col_tx] = pd.to_datetime(tx_fee_df[date_col_tx], format='mixed')
    market_cap_df.set_index(date_col_market, inplace=True)
    tx_fee_df.set_index(date_col_tx, inplace=True)
    
    # Filter data to start from the first valid date after July 1, 2015
    first_valid_date_market_cap = market_cap_df[(market_cap_df.index >= '2015-07-01')].dropna().index.min()
    first_valid_date_tx_fee = tx_fee_df[(tx_fee_df.index >= '2015-07-01')].dropna().index.min()
    market_cap_valid = market_cap_df[market_cap_df.index >= first_valid_date_market_cap]
    tx_fee_valid = tx_fee_df[tx_fee_df.index >= first_valid_date_tx_fee]
    
    # Calculate weekly relative changes
    weekly_relative_market_cap = calculate_weekly_relative_change(market_cap_valid[market_cap_col]).dropna()
    weekly_relative_tx_fee = calculate_weekly_relative_change(tx_fee_valid[tx_fee_col]).dropna()
    
    # Remove infinite values
    weekly_relative_market_cap.replace([np.inf, -np.inf], np.nan, inplace=True)
    weekly_relative_tx_fee.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # Calculate average of the weekly relative changes
    avg_weekly_relative_market_cap = weekly_relative_market_cap.mean()
    avg_weekly_relative_tx_fee = weekly_relative_tx_fee.mean()
    
    ratio = avg_weekly_relative_tx_fee / avg_weekly_relative_market_cap
    
    # Calculate the weights using the formula: w_tx = R / (1 + R)
    w_tx = ratio / (1 + ratio)
    w_balance = 1 - w_tx
    
    return w_tx, w_balance

# Recalculate weights for Bitcoin and Ethereum using the updated function
btc_w_tx, btc_w_balance = calculate_weights(btc_market_cap_new_transformed.copy(), btc_avg_tx_fee_df.copy(), 'date', 'date', 'market_cap', 'avg_tx_fee')
eth_w_tx, eth_w_balance = calculate_weights(eth_market_cap_df.copy(), eth_avg_tx_fee_df.copy(), 'date', 'date', 'MarketCap', 'avg_tx_fee')

btc_w_tx, btc_w_balance, eth_w_tx, eth_w_balance


(0.9799351422949636,
 0.02006485770503641,
 0.9761960435414992,
 0.023803956458500752)

In [478]:
btc_w_tx = 0.95993
btc_w_balance = 0.04006
eth_w_tx = 0.97619
eth_w_balance = 0.02380

# Merging data

In [479]:
# Function to standardize date format while preserving other columns
def standardize_date_format(df, date_column, current_format):
    df[date_column] = pd.to_datetime(df[date_column], format=current_format)
    other_columns = [col for col in df.columns if col != date_column]
    df.set_index(date_column, inplace=True)
    df[other_columns] = df[other_columns]

# Function to resample data to weekly granularity
def resample_to_weekly(df, columns_to_sum):
    return df.resample('W').apply({col: 'sum' for col in columns_to_sum})

# Function to find the closest date
def find_closest_date(target_date, date_series):
    target_date = pd.to_datetime(target_date).value  # Extract int64 representation of the Timestamp
    date_series = pd.to_datetime(date_series.index).values.astype('int64')  # Convert to int64
    kdt = cKDTree(np.array([date_series]).T)
    _, closest_date_idx = kdt.query(np.array([[target_date]]))
    closest_date = pd.to_datetime(date_series[closest_date_idx])
    return closest_date

In [480]:
base_path = './data'
# Load and preprocess your data (use standardize_date_format, resample_to_weekly)
# Load data from CSV and TSV files
btc_emissions = pd.read_csv(f'{base_path}/CCRI/ccri_btc_emissions.csv')
btc_transaction_count = pd.read_csv(f'{base_path}/btc_transaction_count.tsv', delimiter='\t')
btc_supply = pd.read_csv(f'{base_path}/btc_supply.tsv', delimiter='\t')
btc_mock_data = pd.read_csv(f'{base_path}/users/btc_mock_data.csv')

# Standardize date formats
standardize_date_format(btc_emissions, 'date', '%Y-%m-%d')
standardize_date_format(btc_transaction_count, 'date', '%d.%m.%Y')
standardize_date_format(btc_supply, 'date', '%d.%m.%Y')
standardize_date_format(btc_mock_data, 'date', '%m/%d/%Y %H:%M:%S')

# Resample to weekly granularity
btc_emissions_weekly = resample_to_weekly(btc_emissions, ['emissions_24h_mt', 'emissions_365d_mt'])
btc_transaction_count_weekly = resample_to_weekly(btc_transaction_count, ['transaction_count'])
btc_supply_weekly = btc_supply.resample('W').mean()  # Take the last value for supply

# Using SQL-like operations for merging based on the closest dates
# Step 1: Reset index for all dataframes to prepare for SQL-like join
btc_emissions_weekly.reset_index(inplace=True)
btc_transaction_count_weekly.reset_index(inplace=True)
btc_supply_weekly.reset_index(inplace=True)
btc_mock_data.reset_index(inplace=True)

# Step 2: Merge with SQL-like operations
# First, merge btc_mock_data and btc_emissions_weekly
merged_1 = pd.merge_asof(btc_mock_data.sort_values('date'), btc_emissions_weekly.sort_values('date'), 
                         left_on='date', right_on='date', direction='backward')

# Second, merge the result with btc_transaction_count_weekly
merged_2 = pd.merge_asof(merged_1.sort_values('date'), btc_transaction_count_weekly.sort_values('date'), 
                         left_on='date', right_on='date', direction='backward')

# Third, merge the result with btc_supply_weekly
final_btc_personas_emissions = pd.merge_asof(merged_2.sort_values('date'), btc_supply_weekly.sort_values('date'), 
                             left_on='date', right_on='date', direction='backward')

# Show the first few rows of the final merged DataFrame
final_btc_personas_emissions.head()



,date,xpub_address,balance_btc,num_transactions,networth_usd,first_seen,persona_name,emissions_24h_mt,emissions_365d_mt,transaction_count,supply_satoshi
0,2015-11-20 23:58:00,66170,3.058104,0,1000.000000,2015-01-01,The Hodler,0.039347,13.805708,1087244,1.483214e+15
1,2015-11-27 23:58:00,66170,3.058104,0,1092.477064,2015-01-01,The Hodler,0.042570,15.134553,1071224,1.485968e+15
2,2015-12-04 23:58:00,66170,3.058104,0,1186.513761,2015-01-01,The Hodler,0.045045,15.984745,1069889,1.488757e+15
3,2015-12-11 23:58:00,66170,3.058104,0,1335.993884,2015-01-01,The Hodler,0.046762,16.838801,1248549,1.491490e+15
4,2015-12-18 23:58:00,66170,3.058104,0,1410.672783,2015-01-01,The Hodler,0.051966,17.930926,1424724,1.494378e+15


# 4. Emission Attribution
# 4.1 Bitcoin

In [496]:
# Constants for Bitcoin
w_tx_btc = 0.97993  # weight for transaction emissions
w_balance_btc = 0.02006  # weight for balance emissions

# Prepare the DataFrame for calculations
btc_combined_data = final_btc_personas_emissions.copy()

# Calculate share of weekly emissions attributed to each persona
# Isolated Balance Approach
btc_combined_data['share_balance'] = btc_combined_data['balance_btc'] / (btc_combined_data['supply_satoshi'] / 1e8)  # Convert supply from satoshi to BTC
btc_combined_data['emissions_balance_t'] = btc_combined_data['emissions_24h_mt'] * btc_combined_data['share_balance'] * 1000000

# Isolated Transaction Approach
total_weekly_transactions = btc_combined_data['transaction_count']
btc_combined_data['share_tx'] = btc_combined_data['num_transactions'] / total_weekly_transactions
btc_combined_data['emissions_tx_t'] = btc_combined_data['emissions_24h_mt'] * btc_combined_data['share_tx'] * 1000000

# Hybrid Approach
btc_combined_data['share_hybrid'] = w_tx_btc * btc_combined_data['share_tx'] + w_balance_btc * btc_combined_data['share_balance']
btc_combined_data['emissions_hybrid_t'] = btc_combined_data['emissions_24h_mt'] * btc_combined_data['share_hybrid'] * 1000000

# Group by persona to summarize emissions for each approach
btc_persona_summary = btc_combined_data.groupby('persona_name').agg({
    'emissions_balance_t': 'sum',
    'emissions_tx_t': 'sum',
    'emissions_hybrid_t': 'sum',
    'balance_btc': 'mean',
    'num_transactions': 'sum',
    'networth_usd': 'mean',
    'first_seen': 'min'
}).reset_index()

btc_persona_summary


,persona_name,emissions_balance_t,emissions_tx_t,emissions_hybrid_t,balance_btc,num_transactions,networth_usd,first_seen
0,Day Trader,622.810348,605.754318,606.090405,48.847781,2066,7.946545e+05,2016-01-01
1,Institutional Investor,323040.543441,563.048020,7031.940948,18347.759114,1490,4.775758e+08,2018-01-01
2,Occasional User,0.196211,12.819463,12.566112,0.025889,25,8.765961e+02,2020-11-27
3,Retail Payment User,91.401519,579.682866,569.882145,6.718821,1741,1.386536e+05,2017-01-01
4,The Hodler,37.599584,0.000000,0.754248,3.058104,0,4.687002e+04,2015-01-01


In [497]:
btc_combined_data

,date,xpub_address,balance_btc,num_transactions,networth_usd,first_seen,persona_name,emissions_24h_mt,emissions_365d_mt,transaction_count,supply_satoshi,share_balance,emissions_balance_t,share_tx,emissions_tx_t,share_hybrid,emissions_hybrid_t
0,2015-11-20 23:58:00,66170,3.058104,0,1.000000e+03,2015-01-01,The Hodler,0.039347,13.805708,1087244,1.483214e+15,2.061809e-07,0.008113,0.000000e+00,0.000000,4.135988e-09,0.000163
1,2015-11-27 23:58:00,66170,3.058104,0,1.092477e+03,2015-01-01,The Hodler,0.042570,15.134553,1071224,1.485968e+15,2.057988e-07,0.008761,0.000000e+00,0.000000,4.128324e-09,0.000176
2,2015-12-04 23:58:00,66170,3.058104,0,1.186514e+03,2015-01-01,The Hodler,0.045045,15.984745,1069889,1.488757e+15,2.054132e-07,0.009253,0.000000e+00,0.000000,4.120589e-09,0.000186
3,2015-12-11 23:58:00,66170,3.058104,0,1.335994e+03,2015-01-01,The Hodler,0.046762,16.838801,1248549,1.491490e+15,2.050368e-07,0.009588,0.000000e+00,0.000000,4.113038e-09,0.000192
4,2015-12-18 23:58:00,66170,3.058104,0,1.410673e+03,2015-01-01,The Hodler,0.051966,17.930926,1424724,1.494378e+15,2.046406e-07,0.010634,0.000000e+00,0.000000,4.105091e-09,0.000213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1578,2023-06-23 23:58:00,12731,50.898494,9,1.554801e+06,2016-01-01,Day Trader,1.366171,501.615332,2962166,1.940332e+15,2.623185e-06,3.583719,3.038317e-06,4.150862,3.029959e-06,4.139443
1579,2023-06-23 23:58:00,33063,8.700374,4,2.657712e+05,2017-01-01,Retail Payment User,1.366171,501.615332,2962166,1.940332e+15,4.483961e-07,0.612586,1.350363e-06,1.844827,1.332256e-06,1.820090
1580,2023-06-23 23:58:00,66170,3.058104,0,9.341621e+04,2015-01-01,The Hodler,1.366171,501.615332,2962166,1.940332e+15,1.576072e-07,0.215318,0.000000e+00,0.000000,3.161601e-09,0.004319
1581,2023-06-23 23:58:00,32889,212687.064688,6,6.496973e+09,2018-01-01,Institutional Investor,1.366171,501.615332,2962166,1.940332e+15,1.096137e-02,14975.114353,2.025545e-06,2.767241,2.218701e-04,303.112497


In [498]:
df = btc_combined_data[btc_combined_data['persona_name'] == 'The Hodler'] 
df['cum_balance_emissions_kt'] = df['emissions_balance_kt'].cumsum()
px.line(df, x='date', y=['cum_balance_emissions_kt'], title='Bitcoin Emissions')

KeyError: 'emissions_balance_kt'

# 4.2 Ethereum

In [499]:

# Constants for Ethereum
w_tx_eth = 0.97619  # weight for transaction emissions
w_balance_eth = 0.02380  # weight for balance emissions

# Load the Ethereum data
eth_supply = pd.read_csv(f'{base_path}/eth_supply.tsv', delimiter='\t')
eth_emissions_pow = pd.read_csv(f'{base_path}/CCRI/ccri_eth_emissions.csv')
eth_emissions_pos = pd.read_csv(f'{base_path}/CCRI/ccri_eth2_emissions.csv')
eth_mock_data = pd.read_csv(f'{base_path}/users/eth_mock_data.csv')
ethereum_gas_spent = pd.read_csv(f'{base_path}/ethereum_gas_spent.csv')

# Convert PoS emissions to MT (from T)
eth_emissions_pos['emissions_24h_mt'] = eth_emissions_pos['emissions_24h_t'] / 1e6
# Combine PoW and PoS emissions data
eth_emissions = pd.concat([eth_emissions_pow, eth_emissions_pos]).sort_values(by='date')

# Interpret supply as float64
eth_supply['supply_wei'] = eth_supply['supply_wei'].astype('float64')

# Standardize date formats
standardize_date_format(eth_supply, 'date', '%d.%m.%Y')
standardize_date_format(eth_emissions, 'date', '%Y-%m-%d')
standardize_date_format(eth_mock_data, 'date', '%m/%d/%Y %H:%M:%S')
standardize_date_format(ethereum_gas_spent, 'date', '%m/%d/%Y')

# Resample to weekly granularity
eth_supply_weekly = eth_supply.resample('W').mean()  # Take the average value for supply
eth_emissions_weekly = resample_to_weekly(eth_emissions, ['emissions_24h_mt', 'emissions_365d_mt'])
ethereum_gas_spent_weekly = resample_to_weekly(ethereum_gas_spent, ['gasSpent'])

# Merge mock data with emissions, supply, and gas spent data
eth_final_merged = eth_mock_data.copy()
for df in [eth_emissions_weekly, eth_supply_weekly, ethereum_gas_spent_weekly]:
    eth_final_merged = pd.merge_asof(eth_final_merged.sort_values('date'), df.sort_values('date'),
                                     left_on='date', right_on='date', direction='backward')

# Convert 'supply_wei' column to float64 data type (if needed)
eth_final_merged['supply_wei'] = eth_final_merged['supply_wei'].astype('float64')

# Emissions Attribution Calculations
# Isolated Balance Approach
eth_final_merged['share_balance_eth'] = eth_final_merged['balance_eth'] / (eth_final_merged['supply_wei'] / 1e18)
eth_final_merged['emissions_balance_t_eth'] = eth_final_merged['emissions_24h_mt'] * eth_final_merged['share_balance_eth'] * 1000000

# Isolated Transaction Approach
total_weekly_gas_eth = eth_final_merged['gasSpent']
eth_final_merged['share_tx_eth'] = eth_final_merged['gas_spent_addr'] / total_weekly_gas_eth
eth_final_merged['emissions_tx_t_eth'] = eth_final_merged['emissions_24h_mt'] * eth_final_merged['share_tx_eth'] * 1000000

# Hybrid Approach
eth_final_merged['share_hybrid_eth'] = w_tx_eth * eth_final_merged['share_tx_eth'] + w_balance_eth * eth_final_merged['share_balance_eth']
eth_final_merged['emissions_hybrid_t_eth'] = eth_final_merged['emissions_24h_mt'] * eth_final_merged['share_hybrid_eth'] * 1000000

# Summarize by persona
eth_persona_summary = eth_final_merged.groupby('persona_name').agg({
    'emissions_balance_t_eth': 'sum',
    'emissions_tx_t_eth': 'sum',
    'emissions_hybrid_t_eth': 'sum',
    'gas_spent_addr': 'sum',
    'balance_eth': 'mean',
    'networth_usd': 'mean',
    'first_seen': 'min'
}).reset_index()


In [500]:
eth_final_merged

,date,account_address,balance_eth,networth_usd,gas_spent,gas_spent_addr,first_seen,persona_name,emissions_24h_mt,emissions_365d_mt,supply_wei,gasSpent,share_balance_eth,emissions_balance_t_eth,share_tx_eth,emissions_tx_t_eth,share_hybrid_eth,emissions_hybrid_t_eth
0,2016-10-14 23:58:00,0xThe_2164,83.682008,1.000000e+03,NaN,0.0,2015-01-01,The Hodler,0.002488,0.893464,8.400699e+25,18839609062,9.961315e-07,2.478646e-03,0.000000e+00,0.000000,2.370793e-08,0.000059
1,2016-10-14 23:58:00,0xDay_8094,0.000000,0.000000e+00,NaN,130640.0,2016-01-01,Day Trader,0.002488,0.893464,8.400699e+25,18839609062,0.000000e+00,0.000000e+00,6.934326e-06,0.017254,6.769220e-06,0.016844
2,2016-10-21 23:58:00,0xDay_8094,9.384478,1.128953e+02,NaN,88080.0,2016-01-01,Day Trader,0.002661,0.938178,8.421906e+25,10489982322,1.114294e-07,2.965332e-04,8.396582e-06,0.022345,8.199312e-06,0.021820
3,2016-10-21 23:58:00,0xThe_2164,83.682008,1.006695e+03,NaN,0.0,2015-01-01,The Hodler,0.002661,0.938178,8.421906e+25,10489982322,9.936232e-07,2.644206e-03,0.000000e+00,0.000000,2.364823e-08,0.000063
4,2016-10-28 23:58:00,0xDay_8094,73.662492,7.712463e+02,NaN,126956.0,2016-01-01,Day Trader,0.002835,1.009673,8.443122e+25,12769967856,8.724556e-07,2.472998e-03,9.941763e-06,0.028180,9.725815e-06,0.027568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1576,2023-06-23 23:58:00,0xIns_1170,632807.712545,1.188084e+09,NaN,247428.0,2018-01-01,Institutional Investor,0.000044,0.000000,1.192352e+26,754044400757,5.307220e-03,2.314078e-01,3.281345e-07,0.000014,1.266322e-04,0.005521
1577,2023-06-23 23:58:00,0xDAO_1955,7297.963698,1.370178e+07,NaN,1860137.0,2020-01-01,DAO Lending Protocol,0.000044,0.000000,1.192352e+26,754044400757,6.120643e-05,2.668750e-03,2.466880e-06,0.000108,3.864857e-06,0.000169
1578,2023-06-23 23:58:00,0xDay_8094,779.776892,1.464016e+06,NaN,122936.0,2016-01-01,Day Trader,0.000044,0.000000,1.192352e+26,754044400757,6.539819e-06,2.851521e-04,1.630355e-07,0.000007,3.148013e-07,0.000014
1579,2023-06-23 23:58:00,0xRet_4160,399.370345,7.498098e+05,NaN,276654.0,2017-01-01,Retail Payment User,0.000044,0.000000,1.192352e+26,754044400757,3.349432e-06,1.460434e-04,3.668935e-07,0.000016,4.378743e-07,0.000019


# 5. Latex Table

In [501]:
eth_persona_summary

,persona_name,emissions_balance_t_eth,emissions_tx_t_eth,emissions_hybrid_t_eth,gas_spent_addr,balance_eth,networth_usd,first_seen
0,DAO Lending Protocol,1089.078417,61.889167,86.335652,323839691.0,5650.315189,1.061660e+07,2020-01-01
1,Day Trader,211.168732,15.539254,20.195080,68202660.0,721.008953,7.580295e+05,2016-01-01
2,Institutional Investor,8745.055005,11.076993,218.945559,49519397.0,58405.529308,1.050440e+08,2018-01-01
3,Occasional User (PoS),0.000010,0.000170,0.000167,5341689.0,1.121110,1.840571e+03,2022-09-25
4,Retail Payment User,98.374661,14.194692,16.198033,64578113.0,329.836726,3.839002e+05,2017-01-01
5,The Hodler,102.285949,0.319717,2.746510,1433581.0,313.836285,4.610815e+05,2015-01-01


In [538]:
# Assuming btc_persona_summary is the DataFrame containing the Bitcoin persona summary
# and eth_persona_summary is the DataFrame containing the Ethereum persona summary

# Rename columns to specify Bitcoin and Ethereum
btc_persona_summary_table = btc_persona_summary.copy()[['persona_name', 'emissions_balance_t', 'emissions_tx_t', 'emissions_hybrid_t']]
eth_persona_summary_table = eth_persona_summary.copy()[['persona_name', 'emissions_balance_t_eth', 'emissions_tx_t_eth', 'emissions_hybrid_t_eth']]
btc_persona_summary_table.columns = ['Persona Name', 'BTC, Assets, t', 'BTC, Tx, t', 'BTC, Hybrid, t']
eth_persona_summary_table.columns = ['Persona Name', 'ETH, Assets, t', 'ETH, Tx, t', 'ETH, Hybrid, t']

# Merge the two DataFrames on 'Persona Name'
combined_persona_summary = pd.merge(btc_persona_summary_table, eth_persona_summary_table, on='Persona Name', how='outer')


In [545]:
def to_latex_sci_notation(num):
    if pd.isna(num):
        return "--"  # Replace NaN with "--"
    
    elif float(num) <= 0.00001:
        return f"${'0.00001'}$"
    
    elif float(num) <= 0.0001:
        return f"${num:.4}$"
    
    elif float(num) <= 0.001:
        return f"${num:.3}$"
    
    elif float(num) <= 0.01:
        return f"${num:.2}$"
    
    elif float(num) <= 0.1:
        return f"${num:.1}$"
    
    elif float(num) < 1:
        return f"${round(num, 2)}$"
    else:
        return f"${round(num)}$"

# Update the function to generate a LaTeX table with BTC and ETH groups above the column names
# Update the function to generate a LaTeX table with the specified header format
def generate_combined_latex_table(df):
    # Copy the DataFrame and apply the scientific notation function
    latex_df = df.copy()
    for col in df.columns[1:]:
        latex_df[col] = df[col].apply(to_latex_sci_notation)

    # Generate LaTeX table
    latex_code = latex_df.to_latex(index=False, escape=False, na_rep="--", column_format="lcccccc")
    
    # Custom modifications to add midrules and multi-column formatting
    lines = latex_code.splitlines()
    new_lines = []
    for line in lines:
        if line.strip() == r'\toprule':
            new_lines.append(line)
            new_lines.append(r'\multirow{2}{*}{Persona Name} & \multicolumn{3}{c}{BTC} & \multicolumn{3}{c}{ETH} \\')
            new_lines.append(r'\cmidrule(lr){2-4} \cmidrule(lr){5-7}')
        elif 'Persona Name' in line:
            new_lines.append(r'                                      & \textbf{Assets (t)}    & \textbf{Tx (t)}        & \textbf{Hybrid (t)} & \textbf{Assets (t)} & \textbf{Tx (t)} & \textbf{Hybrid (t)} \\')
            new_lines.append(r'\midrule')
        elif line.strip() not in [r'\midrule']:
            new_lines.append(line)
    
            
    return '\n'.join(new_lines)

# Generate LaTeX table
combined_latex_code = generate_combined_latex_table(combined_persona_summary)
print(combined_latex_code)


\begin{tabular}{lcccccc}
\toprule
\multirow{2}{*}{Persona Name} & \multicolumn{3}{c}{BTC} & \multicolumn{3}{c}{ETH} \\
\cmidrule(lr){2-4} \cmidrule(lr){5-7}
                                      & \textbf{Assets (t)}    & \textbf{Tx (t)}        & \textbf{Hybrid (t)} & \textbf{Assets (t)} & \textbf{Tx (t)} & \textbf{Hybrid (t)} \\
\midrule
Day Trader & $623$ & $606$ & $606$ & $211$ & $16$ & $20$ \\
Institutional Investor & $323041$ & $563$ & $7032$ & $8745$ & $11$ & $219$ \\
Occasional User & $0.2$ & $13$ & $13$ & -- & -- & -- \\
Retail Payment User & $91$ & $580$ & $570$ & $98$ & $14$ & $16$ \\
The Hodler & $38$ & $0.00001$ & $0.75$ & $102$ & $0.32$ & $3$ \\
DAO Lending Protocol & -- & -- & -- & $1089$ & $62$ & $86$ \\
Occasional User (PoS) & -- & -- & -- & $1.014e-05$ & $0.00017$ & $0.000167$ \\
\bottomrule
\end{tabular}


# 5.1 Persona metrics

In [556]:
combined_persona_summary = pd.merge(btc_persona_summary[['persona_name','first_seen', 'balance_btc', 'num_transactions']], eth_persona_summary[['persona_name','first_seen','balance_eth', 'gas_spent_addr', 'networth_usd']], on='persona_name', how='outer', suffixes=('_btc', '_eth'))

In [557]:
combined_persona_summary

,persona_name,first_seen_btc,balance_btc,num_transactions,first_seen_eth,balance_eth,gas_spent_addr,networth_usd
0,Day Trader,2016-01-01,48.847781,2066.0,2016-01-01,721.008953,68202660.0,7.580295e+05
1,Institutional Investor,2018-01-01,18347.759114,1490.0,2018-01-01,58405.529308,49519397.0,1.050440e+08
2,Occasional User,2020-11-27,0.025889,25.0,NaN,NaN,NaN,NaN
3,Retail Payment User,2017-01-01,6.718821,1741.0,2017-01-01,329.836726,64578113.0,3.839002e+05
4,The Hodler,2015-01-01,3.058104,0.0,2015-01-01,313.836285,1433581.0,4.610815e+05
5,DAO Lending Protocol,NaN,NaN,NaN,2020-01-01,5650.315189,323839691.0,1.061660e+07
6,Occasional User (PoS),NaN,NaN,NaN,2022-09-25,1.121110,5341689.0,1.840571e+03


In [566]:
def to_latex_sci_notation_persona(num):

    if pd.isna(num):
        return "--"  # Replace NaN with "--"
    try:
        num = float(num)
    except ValueError:
        return num  # If conversion to float fails, return the original string (useful for dates)
    
    if num < 1:
        return f"${round(num, 2)}$"
    if num < 100000:
        return f"${round(num)}$"
    else:
        return f"${num:.2e}$"

def generate_persona_metric_table(df):
    # Create a single 'First Seen' column that takes either 'first_seen_btc' or 'first_seen_eth', whichever is available
    df['First Seen'] = df['first_seen_btc'].fillna(df['first_seen_eth'])
    
    # Reorganize columns to place 'First Seen' next to 'Persona Name'
    columns = ['persona_name', 'First Seen', 'balance_btc', 'num_transactions', 'balance_eth', 'gas_spent_addr', 'networth_usd']
    df = df[columns]
    
    # Copy the DataFrame and apply the scientific notation function
    latex_df = df.copy()
    for col in df.columns[2:]:
        latex_df[col] = df[col].apply(to_latex_sci_notation)

    # Generate LaTeX table
    latex_code = latex_df.to_latex(index=False, escape=False, na_rep="--", column_format="lcccccc")
    
    # Custom modifications to add midrules and multi-column formatting
    lines = latex_code.splitlines()
    new_lines = []
    for line in lines:
        if line.strip() == r'\toprule':
            new_lines.append(line)
            new_lines.append(r'Persona Name & \textbf{First Seen} & \multicolumn{2}{c}{Bitcoin} & \multicolumn{3}{c}{Ethereum} \\')
            new_lines.append(r'\cmidrule(lr){3-4} \cmidrule(lr){5-7}')
        elif 'persona_name' in line:
            new_lines.append(r'                         &            & Balance BTC               & Transaction Count & Balance ETH & Gas Spent & Net Worth (USD) \\')
            new_lines.append(r'\midrule')
        elif line.strip() not in [r'\midrule', r'\bottomrule']:
            new_lines.append(line)
    new_lines.append(r'\bottomrule')
            
    return '\n'.join(new_lines)


            

combined_latex_code = generate_persona_metric_table(combined_persona_summary)
print(combined_latex_code)


\begin{tabular}{lccccccc}
\toprule
\multirow{2}{*}{Persona Name} & \multicolumn{2}{c}{First Seen} & \multicolumn{2}{c}{Bitcoin} & \multicolumn{3}{c}{Ethereum} \\
\cmidrule(lr){2-3} \cmidrule(lr){4-5} \cmidrule(lr){6-8}
                                  & BTC       & ETH        & Balance BTC               & Transaction Count & Balance ETH & Gas Spent & Net Worth (USD) \\
\midrule
Day Trader & 2016-01-01 & $49$ & $2066$ & $721$ & $6.82e+07$ & $7.58e+05$ \\
Institutional Investor & 2018-01-01 & $18348$ & $1490$ & $58406$ & $4.95e+07$ & $1.05e+08$ \\
Occasional User & 2020-11-27 & $0.03$ & $25$ & -- & -- & -- \\
Retail Payment User & 2017-01-01 & $7$ & $1741$ & $330$ & $6.46e+07$ & $3.84e+05$ \\
The Hodler & 2015-01-01 & $3$ & $0.0$ & $314$ & $1.43e+06$ & $4.61e+05$ \\
DAO Lending Protocol & 2020-01-01 & -- & -- & $5650$ & $3.24e+08$ & $1.06e+07$ \\
Occasional User (PoS) & 2022-09-25 & -- & -- & $1$ & $5.34e+06$ & $1841$ \\
\bottomrule
\end{tabular}


In [513]:
eth_[e]

NameError: name 'eth_' is not defined